In [44]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys
import datetime
from pandas.tseries.offsets import *

In [2]:
#this function is used to update data that didn't populate in initial data call above
def update_data(symbol):
    data = yf.download(
            tickers = symbol,
            period = '200d',
            group_by = 'ticker',
            auto_adjust = True,
            threads = True,
            proxy = None)
    return data

In [57]:
stocks = pd.read_csv('cleaned_main_data.csv')
stocks.head()

,Unnamed: 0,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date
0,0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31 00:00:00
1,1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,NaN,NaN,1.477958e+09,1000:801,2020-10-14,2020-06-30 00:00:00
2,2,AAN,"Aaron's, Inc",NYSE,Rental & Leasing Services,Industrials,0.16,1.600214e+09,1.271376e+09,3:2,2020-11-02,2020-06-30 00:00:00
3,3,AAP,"Advance Auto Parts, Inc W/I",NYSE,Specialty Retail,Consumer Cyclical,1.00,1.600301e+09,1.127693e+09,3:2,2020-11-10,2020-07-11 00:00:00
4,4,AAT,"American Assets Trust, Inc",NYSE,REIT—Diversified,Real Estate,1.00,1.599610e+09,NaN,NaN,2020-10-27,2020-06-30 00:00:00


In [4]:
stocks = stocks.drop(stocks.columns[0], axis=1)

In [5]:
# symbols = stocks['Symbol'][:5].tolist()
symbols = stocks['Symbol'].tolist()
type(symbols)

list

In [6]:
len(symbols)

3940

In [7]:
data = yf.download(
tickers = symbols,
period = '200d',
group_by = 'ticker',
auto_adjust = True,
threads = True,
proxy = None)

[*********************100%***********************]  3934 of 3934 completed

6 Failed downloads:
- GBIO: No data found, symbol may be delisted
- NATI: No data found, symbol may be delisted
- GIFI: No data found, symbol may be delisted
- FWONA: No data found, symbol may be delisted
- THR: No data found, symbol may be delisted
- GFED: No data found, symbol may be delisted


In [8]:
data.head(2)

ETB                                              CVBF  \
                 Open       High       Low      Close Volume       Open   
Date                                                                      
2019-11-18  15.490252  15.508671  15.37053  15.453415  46900  20.706866   
2019-11-19  15.434990  15.545502  15.42578  15.499455  39000  20.726316   

                                                     ...         AAP  \
                 High        Low      Close  Volume  ...        Open   
Date                                                 ...               
2019-11-18  20.706866  20.512343  20.629055  255200  ...  161.767930   
2019-11-19  20.804125  20.609604  20.658234  358000  ...  162.135468   

                                                         REAL             \
                  High         Low       Close  Volume   Open       High   
Date                                                                       
2019-11-18  162.969882  159.880546  162.582474  862900  17.66  18.184999   
2019-11-19  162.532804  160.148757  160.903702  714500  17.25  17.639999   

                                      
              Low      Close  Volume  
Date                                  
2019-11-18  16.90  17.240000  943300  
2019-11-19  16.65  17.549999  633700  

[2 rows x 19676 columns]

In [9]:
stocks['new_date'] = stocks.previous_earnings_date.map(lambda x: x + 0*BDay())
stocks.previous_earnings_date = pd.to_datetime(stocks.previous_earnings_date, errors='coerce')

(3940, 11)

In [10]:
new_stocks = stocks[~(stocks.Symbol.isin(['GABC', 'FWONA', 'GFED', 'NATI']))]

In [11]:
new_stocks.shape

(3936, 11)

In [12]:
stocks.head(2)

,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date
0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31 00:00:00
1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,NaN,NaN,1.477958e+09,1000:801,2020-10-14,2020-06-30 00:00:00


In [72]:
earnings_date_index = stocks.columns.tolist().index('new_date')
symbol_index = stocks.columns.tolist().index('Symbol')

In [74]:
indicators = ['BBP', 'Exponential Moving Average']
for indicator in indicators:
    stocks.loc[:, indicator] = np.nan
stocks.head(1)

,Unnamed: 0,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date,new_date,BBP,Exponential Moving Average
0,0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31,2020-07-31,NaN,NaN


In [75]:
#calculate exponential moving average
def EMA(data, date, days=5, window=20, smoothing=2):
    data = data / data.iloc[0]
    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['EMA'] = data['SMA']
    
    ema_index = data.columns.tolist().index('EMA')
    value_index = data.columns.tolist().index('Close')
    
    i=0
    for row in data.itertuples():
        
        if i == 0:
            previous_row = row
        else:
#             try:
#                 value_today = row[value_index+1]
#                 ema = (value_today *(smoothing/(1.0+days)) 
#                        + previous_row[ema_index+1] *(1.0-(smoothing/(1.0+days))))
#                 data.at[i,'EMA'] = ema
                
#             except:
#                 e = sys.exc_info()[0]
            value_today = row[value_index+1]
            ema = (value_today *(smoothing/(1.0+days)) 
                   + previous_row[ema_index+1] *(1.0-(smoothing/(1.0+days))))
            data.at[row[0],'EMA'] = ema
            previous_row = row
            
        i += 1
#     print(data)
    idx = data.index.get_loc(date)
    ema = data['EMA'].iloc[idx-days]
#     print(bbp)
    return ema


In [76]:
#calculate Bollinger Bands
def BBP(data, date, days=5, window=20):

    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['SD'] = data['Close'].rolling(window=window).std()
    data['BB_above'] = data['SMA'] + 2*data['SD']
    data['BB_below'] = data['SMA'] - 2*data['SD']
    data['BBP'] = np.divide(data['Close'] - data['SMA'], 2*data['SD'])

    #get the index of the earnings date
    idx = data.index.get_loc(date)
    
    #
#     print(data[idx-days:idx])
    bbp = data['BBP'].iloc[idx-days]
#     print(bbp)
    return bbp
    
#     print (data['BBP']index)
#     return data['BBP'].loc[date]
#     data[symbol]['Open'].loc[earnings_date]

In [77]:
#calculate momentum
def mom(data, date, days=5, window=20):
    pass
#     data = data / data.iloc[0]
#     data['Close'] = np.divide(data.iloc[window:, 'Close'], data.iloc[::data.shape[0]-window, 'Close'])-1
#     print(data)
    
#     pass

In [85]:
def nat_check(nat):
    return nat == np.datetime64('NaT')  

In [ ]:
index=0
days=5

for row in stocks.itertuples():
    earnings_date=(row[earnings_date_index+1])
    symbol = row[symbol_index+1]
#     print (data[symbol]['Open'].loc[a])
    
    # some data wasn't populated.  Check for this and populate it if so
    j=0
    
#     print(earnings_date)
    if not pd.isnull(earnings_date):
        
        try:
            while np.isnan(data[symbol]['Open'].loc[earnings_date]):

                data[symbol] = update_data(symbol)
                j += 1
                if j > 5:
                    print('Data for ', symbol, ' will not update!')
                    break

        #get BBP.  Using try because data may not have updated above!!!
            stocks.at[index, 'Exponential Moving Average'] == EMA(data[symbol].copy(), earnings_date, days)

    #     for indicator in indicators:
    #         try:
    #             if indicator == 'BBP':
    #                 stocks.at[index,indicator] = BBP(data[symbol].copy(), earnings_date, days)
    #             elif indicator == 'Momentum':
    #                 stocks.at[index,indicator] == mom(data[symbol].copy(), earnings_date, days)
    #             else:
    #                 stocks.at[index,indicator] == EMA(data[symbol].copy(), earnings_date, days)
    #         except:
    #             print('Could not create ', indicator, ' for ', symbol)
        except:
            e = sys.exc_info()[0]
            print (e)
    
#     print(initial_date)
#     print (symbol)
#     print (row)
    index +=1
#     break

2020-07-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-07-13 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-31 00:00:00
2020-06-22 00:00:00
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-05-04 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00


[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
NaT
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-31 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-07-03 00:00:00
<class 'KeyError'>
2020-03-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-08-03 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-05-04 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00

2020-04-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-08-03 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
NaT
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-01 00:00:00
2020-06-01 00:00:00
2020-06-01 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
NaT
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-07-06 00:00:00
2020-06-30 00:00:00
2020-08-03 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-07-13 00:00:00
2020-06-29 00:

[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-01-31 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-01-31 00:00:00
2020-01-31 00:00:00
2019-12-31 00:00:00
2019-12-31 00:00:00
2019-12-31 00:00:00
2020-08-03 00:00:00
2020-06-29 00:00:00
2020-06-29 00:00:00
2020-07-03 00:00:00
<class 'KeyError'>
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-

[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-02 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%*********

[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-07-31 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00

[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-03-02 00:00:00
2020-07-31 00:00:00
2020-05-04 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-02 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-07-06 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-03-02 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-03-02 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-03-31 00:00:00
[*********************100%****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data for  ANNX  will not update!
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-01 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2019-12-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-06-30 00

[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-05-11 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-29 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-29 00:00:00
2020-03-31 00:00:00
2020-06-30 00:00:00
2020-07-31 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-04-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00:00:00
2020-06-30 00

In [ ]:
stocks.head(5)

In [27]:
datetime.date.weekday('2020-07-11')

TypeError: descriptor 'weekday' for 'datetime.date' objects doesn't apply to a 'str' object

In [40]:
date = datetime.datetime.strptime("20/07/11", "%y/%m/%d")

In [37]:
dt = datetime.datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M")

In [41]:
date.weekday()

5

In [43]:
stocks.previous_earnings_date

0       2020-07-31 00:00:00
1       2020-06-30 00:00:00
2       2020-06-30 00:00:00
3       2020-07-11 00:00:00
4       2020-06-30 00:00:00
               ...         
3935    2020-06-30 00:00:00
3936                   Open
3937    2020-06-30 00:00:00
3938    2020-04-30 00:00:00
3939    2020-06-30 00:00:00
Name: previous_earnings_date, Length: 3940, dtype: object

In [47]:
stocks.head()

,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date,BBP,Exponential Moving Average
0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31 00:00:00,NaN,NaN
1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,NaN,NaN,1.477958e+09,1000:801,2020-10-14,2020-06-30 00:00:00,NaN,NaN
2,AAN,"Aaron's, Inc",NYSE,Rental & Leasing Services,Industrials,0.16,1.600214e+09,1.271376e+09,3:2,2020-11-02,2020-06-30 00:00:00,NaN,NaN
3,AAP,"Advance Auto Parts, Inc W/I",NYSE,Specialty Retail,Consumer Cyclical,1.00,1.600301e+09,1.127693e+09,3:2,2020-11-10,2020-07-11 00:00:00,NaN,NaN
4,AAT,"American Assets Trust, Inc",NYSE,REIT—Diversified,Real Estate,1.00,1.599610e+09,NaN,NaN,2020-10-27,2020-06-30 00:00:00,NaN,NaN


In [53]:
stocks.previous_earnings_date.dtype

dtype('<M8[ns]')

In [55]:
np.where(stocks.previous_earnings_date.dt.weekday_name.isin(['satruday', 'Sunday']), stocks.previous_earnings_date+BDay(1), stocks.previous_earnings_date)

AttributeError: 'DatetimeProperties' object has no attribute 'weekday_name'

In [62]:
stocks.previous_earnings_date.unique()

array(['2020-07-31T00:00:00.000000000', '2020-06-30T00:00:00.000000000',
       '2020-07-11T00:00:00.000000000', '2020-04-30T00:00:00.000000000',
       '2020-03-31T00:00:00.000000000', '2020-06-20T00:00:00.000000000',
       '2020-05-31T00:00:00.000000000', '2020-05-02T00:00:00.000000000',
       '2019-12-31T00:00:00.000000000', '2020-06-28T00:00:00.000000000',
       '2020-06-26T00:00:00.000000000', '2020-06-27T00:00:00.000000000',
                                 'NaT', '2020-05-09T00:00:00.000000000',
       '2020-02-29T00:00:00.000000000', '2020-01-31T00:00:00.000000000',
       '2020-08-01T00:00:00.000000000', '2019-10-31T00:00:00.000000000',
       '2019-08-31T00:00:00.000000000', '2019-06-30T00:00:00.000000000',
       '2020-07-03T00:00:00.000000000', '2020-05-03T00:00:00.000000000',
       '2020-04-26T00:00:00.000000000', '2020-08-02T00:00:00.000000000',
       '2020-06-14T00:00:00.000000000', '2020-07-25T00:00:00.000000000',
       '2020-06-24T00:00:00.000000000', '2020-07-05

In [64]:
stocks['New']=np.where(stocks.previous_earnings_date.dt.day_name.isin(
    ['Saturday','Sunday']),stocks.previous_earnings_date+BDay(1),stocks.previous_earnings_date)


AttributeError: 'function' object has no attribute 'isin'

In [66]:
stocks.previous_earnings_date.dt.day_name.isin()

AttributeError: 'function' object has no attribute 'isin'

In [68]:
stocks['new_date'] = stocks.previous_earnings_date.map(lambda x: x + 0*BDay())

In [71]:
stocks.head()

,Unnamed: 0,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date,new_date
0,0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31,2020-07-31
1,1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,NaN,NaN,1.477958e+09,1000:801,2020-10-14,2020-06-30,2020-06-30
2,2,AAN,"Aaron's, Inc",NYSE,Rental & Leasing Services,Industrials,0.16,1.600214e+09,1.271376e+09,3:2,2020-11-02,2020-06-30,2020-06-30
3,3,AAP,"Advance Auto Parts, Inc W/I",NYSE,Specialty Retail,Consumer Cyclical,1.00,1.600301e+09,1.127693e+09,3:2,2020-11-10,2020-07-11,2020-07-13
4,4,AAT,"American Assets Trust, Inc",NYSE,REIT—Diversified,Real Estate,1.00,1.599610e+09,NaN,NaN,2020-10-27,2020-06-30,2020-06-30
